In [22]:
import shutil
import matplotlib.pyplot as plt
import requests
import time

import micasense.metadata as metadata
import os,glob

In [92]:
start_time = time.time()
elapsed_time = time.time() - start_time
print("Elapsed time: %.10f seconds." % elapsed_time)  

Elapsed time: 1.4373488426 seconds.


In [25]:
#!/usr/bin/env python

##################################################################################
# The MIT License (MIT)
# 
# Copyright (c) 2015 MicaSense, Inc.
# 
##################################################################################

#Read the GPS data from the camera
gps_data = requests.get('http://192.168.10.254/gps')
print("Raw data : ", str(gps_data.text))
print("Has 3d fix: ",str(gps_data.json()['fix3d']))


#Configure parameter to the camera
capture_params = { 'enabled_bands_raw' : 31}
capture_data = requests.post("http://192.168.10.254/config", json=capture_params)


#Post a message to the camera commanding a capture, block until complete
capture_params = { 'store_capture' : True, 'block' : True }
capture_data = requests.post("http://192.168.10.254/capture", json=capture_params)
print(capture_data.json())


Raw data :  {"altitude":2552.207,"fix3d":true,"latitude":0.08195555309493312,"longitude":-1.293311265327675,"p_acc":0,"utc_time":"2020-10-30T15:10:36.999568Z","v_acc":5.657999992370605,"vel_d":0.17,"vel_e":0,"vel_n":0.05}

Has 3d fix:  True
{'id': 'DQrOWlIgqlfgFbp1TdNm', 'jpeg_cache_path': {'1': '/images/tmp26.jpg', '2': '/images/tmp25.jpg', '3': '/images/tmp27.jpg', '4': '/images/tmp28.jpg', '5': '/images/tmp29.jpg'}, 'jpeg_storage_path': {'1': '/files/0016SET/000/IMG_0002_1.jpg', '2': '/files/0016SET/000/IMG_0002_2.jpg', '3': '/files/0016SET/000/IMG_0002_3.jpg', '4': '/files/0016SET/000/IMG_0002_4.jpg', '5': '/files/0016SET/000/IMG_0002_5.jpg'}, 'raw_cache_path': {'1': '/images/tmp20.tif', '2': '/images/tmp21.tif', '3': '/images/tmp22.tif', '4': '/images/tmp23.tif', '5': '/images/tmp24.tif'}, 'raw_storage_path': {'1': '/files/0016SET/000/IMG_0002_1.tif', '2': '/files/0016SET/000/IMG_0002_2.tif', '3': '/files/0016SET/000/IMG_0002_3.tif', '4': '/files/0016SET/000/IMG_0002_4.tif', '5': 

In [30]:
datStr = str(capture_data.json()['jpeg_cache_path']['5'])

data = requests.get('http://192.168.10.254{}'.format(datStr), stream=True)
## option 1
with open('IMG_0002_5.jpg', 'wb') as out_file:
    shutil.copyfileobj(data.raw, out_file)
del data


In [111]:
# # Image from the example RedEdge imageSet (see the ImageSet notebook) without RigRelatives.
imagePath = os.path.expanduser(os.path.join('.'))
imageNames = glob.glob(os.path.join(imagePath,'IMG_0007_*.jpg'))
#panelNames = glob.glob(os.path.join(imagePath,'000','IMG_0000_*.tif'))


In [36]:
from __future__ import print_function
import cv2
import numpy as np


MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15


def alignImages(im1, im2):

  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
  
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
  
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)
  
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)

  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]

  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)
  
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)

  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
  
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
  
  return im1Reg, h


if __name__ == '__main__':
  
  # Read reference image
    refFilename = "IMG_0002_1.jpg"
    print("Reading reference image : ", refFilename)
    imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)

  # Read image to be aligned
    imFilename = "IMG_0002_2.jpg"
    print("Reading image to align : ", imFilename);  
    im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
  
    print("Aligning images ...")
  # Registered image will be resotred in imReg. 
  # The estimated homography will be stored in h. 
    imReg, h = alignImages(im, imReference)
  
  # Write aligned image to disk. 
    outFilename = "aligned2.jpg"
    print("Saving aligned image : ", outFilename); 
    cv2.imwrite(outFilename, imReg)

  # Print estimated homography
    print("Estimated homography : \n",  h)
  

Reading reference image :  IMG_0002_1.jpg
Reading image to align :  IMG_0002_2.jpg
Aligning images ...
Saving aligned image :  aligned2.jpg
Estimated homography : 
 [[ 1.24503448e+00 -8.11844737e-02  6.95446890e+00]
 [ 3.50126001e-01  1.25155962e+00 -9.77242866e+01]
 [ 4.00498420e-04  1.22223854e-04  1.00000000e+00]]


In [1]:
class ProyectividadOpenCV():
    """
    Esta es una clase para solucionar problemas con homografias
    """
    # Atributos de la clase
    error_reproyeccion = 4
    #--------------------------------------------------------------------------    
    def __init__(self):
        """Inicializador del objeto miembro de la clase"""
        
    
    #--------------------------------------------------------------------------    
    def coser_imagenes(self, ruta_img_base, ruta_img_adicional, radio = 0.75, error_reproyeccion = 4.0, coincidencias = False):
        """Metodo que carga una imagen desde una ruta en disco duro"""
        
        imagen_adicional = ruta_img_adicional
        imagen_base = ruta_img_base
        # Se obtienen los puntos deinterés
        
        (kpsBase, featuresBase) = self.obtener_puntos_interes(imagen_base)
        (kpsAdicional, featuresAdicional) = self.obtener_puntos_interes(imagen_adicional)
        # Se buscan las coincidencias
        
        M = self.encontrar_coincidencias(imagen_base, imagen_adicional, kpsBase, kpsAdicional, featuresBase, featuresAdicional, radio)
        
        if M is None:
            return None
        
        # Se halla la homgrafia
        (H, status) = self.encontrar_H_RANSAC(M, kpsBase, kpsAdicional, error_reproyeccion)
              
        # Organizando la imagen resultante
        
        result = cv2.warpPerspective(imagen_base, H, (imagen_base.shape[1], imagen_base.shape[0]))
        result[0:imagen_adicional.shape[0], 0:imagen_adicional.shape[1]] = imagen_adicional

        # check to see if the keypoint matches should be visualized
        if coincidencias:
            vis = self.dibujar_coincidencias(imagen_base, imagen_adicional, kpsBase, kpsAdicional, M, status)

            # return a tuple of the stitched image and the
            # visualization
            return (result, vis)

        # return the stitched image
        return result
    
    #--------------------------------------------------------------------------
    def estabilizador_imagen(self, imagen_base, imagen_a_estabilizar, radio = 0.75, error_reproyeccion = 4.0, coincidencias = False):
        """Esta clase devuelve una secuencia de imágenes tomadas de la cámara estabilizada con respecto a la primera imagen"""
        
        # Se obtienen los puntos deinterés
        
        (kpsBase, featuresBase) = self.obtener_puntos_interes(imagen_base)
        (kpsAdicional, featuresAdicional) = self.obtener_puntos_interes(imagen_a_estabilizar)
        # Se buscan las coincidencias        
        
        M = self.encontrar_coincidencias(imagen_base, imagen_a_estabilizar, kpsBase, kpsAdicional, featuresBase, featuresAdicional, radio)
        
        if M is None:
            print("pocas coincidencias")
            return None
        
        if len(M) > 4:
            # construct the two sets of points
            
#            M2 = cv2.getPerspectiveTransform(ptsA,ptsB)
            (H, status) = self.encontrar_H_RANSAC_Estable(M, kpsBase, kpsAdicional, error_reproyeccion)
            estabilizada = cv2.warpPerspective(imagen_base,H,(imagen_base.shape[1],imagen_base.shape[0]))
            return estabilizada
        print("sin coincidencias")
        return None
        
            
     
    #--------------------------------------------------------------------------
    def obtener_puntos_interes(self, imagen):
        """Se obtienen los puntos de interes cn SIFT"""
        
        descriptor = cv2.xfeatures2d.SIFT_create()
        (kps, features) = descriptor.detectAndCompute(imagen, None)
        
        return kps, features
    
    #--------------------------------------------------------------------------
    def encontrar_coincidencias(self, img1, img2, kpsA, kpsB, featuresA, featuresB, ratio):
        """Metodo para estimar la homografia"""
        
        matcher = cv2.DescriptorMatcher_create("BruteForce")
        rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
        matches = []
#        
#        # loop over the raw matches
        for m in rawMatches:
#            # ensure the distance is within a certain ratio of each
#            # other (i.e. Lowe's ratio test)
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                matches.append((m[0].trainIdx, m[0].queryIdx))
        
#        print (matches)
        return matches
    
    #--------------------------------------------------------------------------
    def encontrar_H_RANSAC(self, matches, kpsA, kpsB, reprojThresh):
        """Metodo para aplicar una H a una imagen y obtener la proyectividad"""

        if len(matches) > 4:
            # construct the two sets of points
            ptsA = np.float32([kpsA[i].pt for (_, i) in matches])
            ptsB = np.float32([kpsB[i].pt for (i, _) in matches])

            # compute the homography between the two sets of points
            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)

            # return the matches along with the homograpy matrix
            # and status of each matched point
            return (H, status)

        # otherwise, no homograpy could be computed
        return None
    
    
    #--------------------------------------------------------------------------
    def encontrar_H_RANSAC_Estable(self, matches, kpsA, kpsB, reprojThresh):
        """Metodo para aplicar una H a una imagen y obtener la proyectividad"""
        
        if len(matches) > 4:
            # construct the two sets of points
            ptsA = np.float32([kpsA[i].pt for (_, i) in matches])
            ptsB = np.float32([kpsB[i].pt for (i, _) in matches])
    
            # compute the homography between the two sets of points
            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)
            
            return (H, status)

        return None

    def dibujar_coincidencias(self, imagen_base, imagen_adicional, kpsA, kpsB, matches, status):
        
        (hA, wA) = imagen_base.shape[:2]
        (hB, wB) = imagen_adicional.shape[:2]
        vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
        vis[0:hA, 0:wA] = imagen_base
        vis[0:hB, wA:] = imagen_adicional

        # loop over the matches
        for ((trainIdx, queryIdx), s) in zip(matches, status):
            # only process the match if the keypoint was successfully
            # matched
            if s == 1:
                # draw the match
                ptA = (int(kpsA[queryIdx].pt[0]), int(kpsA[queryIdx].pt[1]))
                ptB = (int(kpsB[trainIdx].pt[0]) + wA, int(kpsB[trainIdx].pt[1]))
                cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

        # return the visualization
        return vis
    

    #--------------------------------------------------------------------------
    def encontrar_H_marcas(self, las_xprima, las_x):
        """Metodo para estimar la homografia"""
        #Se utiliza 0 y no RANSAC porque deseo que utilice todos los puntos que se tienen
        H, estado = cv2.findHomography(las_x, las_xprima, 0,0.1)
        return H, estado

    #--------------------------------------------------------------------------
    def estabilizar_desde_marcas(self, imagen, marcas_click, marcas_cad_mm):
        """Esta clase retorna una imagen estabilizada con base en una imagen abstraida delas marcas del cad dadas en mm"""
        
        #Lo primero es tratar la imagen entrante
        blur = cv2.blur(imagen, (3,3))
        hsv = cv2.cvtColor(blur,cv2.COLOR_BGR2HSV)
        
        #Se aplica un filtro de color verde para separar las marcas del fondo
        thresh_marcas = cv2.inRange(hsv,np.array((49,50,50)), np.array((107, 255, 255)))
        marcas_H = list()
        cad_H = list()
        #Se hace una busqueda de las marcas visibles en un radio de 30 pixelesy se filtranpor area para sacar los pares que permitiran hallar la homografia
        for i in range(0,len(marcas_click)):
#            print(i)
            x_men=marcas_click[i][0]-10
            x_may=marcas_click[i][0]+10
            y_men=marcas_click[i][1]-10
            y_may=marcas_click[i][1]+10
            area_marca = thresh_marcas[y_men:y_may, x_men:x_may]
            image_marcas, contours_marcas,hierarchy_marcas = cv2.findContours(area_marca,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
            
            max_area = 65
            best_cnt = 1
            for cnt in contours_marcas:
                area = 1
                area = cv2.contourArea(cnt)
#                print(contours_marcas)
#                print(area,m)
                if area > max_area and area < 85:
                    max_area = area
                    best_cnt = cnt
                    # finding centroids of best_cnt and draw a circle there
                    cM = cv2.moments(best_cnt)
                    cx,cy = int(cM['m10']/cM['m00']), int(cM['m01']/cM['m00'])
                    cx = x_men+cx
                    cy = y_men+cy
                    marcas_H.append((cx,cy))
                    cad_H.append((marcas_cad_mm[i][0]+100,marcas_cad_mm[i][1]))
#                    print(marcas_H,cad_H)
                    
                    
        las_x = np.array(cad_H)
        las_xprima = np.array(marcas_H)
        print(las_x,las_xprima)
        if len(las_xprima) > 4:
            
            H, estado = self.encontrar_H_marcas(las_x, las_xprima)

            estabilizada = cv2.warpPerspective(imagen,H,(1200,1200))
            
            return estabilizada
        
        return None
    
    #--------------------------------------------------------------------------
    def estabilizar_desde_centroides_marcas(self, imagen, marcas_click, marcas_cad_mm):
        """Esta clase retorna una imagen estabilizada con base en una imagen abstraida delas marcas del cad dadas en mm"""
        
        las_x = np.array(marcas_cad_mm)
        las_xprima = np.array(marcas_click)
        print(las_x,las_xprima)
        if len(las_xprima) > 4:
            
            H, estado = self.encontrar_H_marcas(las_x, las_xprima)

            estabilizada = cv2.warpPerspective(imagen,H,(650,650))
            
            return estabilizada
        
        return None
            
            
    #--------------------------------------------------------------------------
    def inicializar_marcas(self, img_base):
        """Permite al usuario hacer click en el centro apriximado de cada marca y las guarda en orden"""
        
        global puntos_click
        
        #Copiar la imagen original para poder escribir sobre ella
        #Sin modificarla
        imagen_conmarcas =self.img_base.copy()
        
        #Mostrar la imagen
        cv2.namedWindow("Imagen_base")
        cv2.setMouseCallback("Imagen_base", click_and_count)
        
        while True:
            # Mostrar a imagen
            cv2.imshow("Imagen_base", imagen_conmarcas)
            key = cv2.waitKey(1) & 0xFF
            
            # Menu principal
            #Si se presiona r resetee la imagen
            if key == ord("r"):
                puntos_click = list()
                imagen_conmarcas = self.img_base.copy()
                
            # Si se presiona q salir
            elif key == ord("q"):
                return puntos_click
                break
        
            # Graficar los puntos que hayan en puntos_click
            if puntos_click:
                for pts_id, coords in enumerate(puntos_click):
                    #Coordenadas
                    x, y = coords[0], coords[1]
                    # Dibujar un circulo
                    cv2.circle(imagen_conmarcas, (x, y), 5, (0,0,255), 5, 2)
                    # Seleccionar una fuente
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(imagen_conmarcas, str(pts_id+1), (x, y), font, 10, (0,0,255),5)
 


    def img_alignment_sequoia(self, img_RGB, img_GRE, img_base_NIR, img_RED, img_REG, width, height):
        """This class takes the five images given by Sequoia Camera and makes a photogrammetric
        alignment. Returns four images (GRE, NIR, RED, REG) aligned with the RGB image"""

        # Makes resize to all images

        b_RGB = cv2.resize(img_RGB, (width, height), interpolation=cv2.INTER_LINEAR)
        b_GRE = cv2.resize(img_GRE, (width, height), interpolation=cv2.INTER_LINEAR)
        base_NIR = cv2.resize(img_base_NIR, (width, height), interpolation=cv2.INTER_LINEAR)
        b_RED = cv2.resize(img_RED, (width, height), interpolation=cv2.INTER_LINEAR)
        b_REG = cv2.resize(img_REG, (width, height), interpolation=cv2.INTER_LINEAR)

        # Makes a stabilization with NIR image like base

        stb_GRE = self.estabilizador_imagen(b_GRE, base_NIR)
        stb_RGB = self.estabilizador_imagen(b_RGB, base_NIR)
        stb_RED = self.estabilizador_imagen(b_RED, base_NIR)
        stb_REG = self.estabilizador_imagen(b_REG, base_NIR)

        return stb_RGB, stb_GRE, base_NIR, stb_RED, stb_REG   
#===========================================================================        
def main():
    """Computer Vision Class"""
    print("Computer Vision Class by Jorge Martinez")

#===========================================================================
if __name__ == '__main__':
    main()
        

Computer Vision Class by Jorge Martinez


In [1]:
class ProyectividadOpenCV():
    """
    Esta es una clase para solucionar problemas con homografias
    """
    # Atributos de la clase
    error_reproyeccion = 4
    #--------------------------------------------------------------------------    
    def __init__(self):
        """Inicializador del objeto miembro de la clase"""
        
    
    #-------------------------------------------------------------------------- 
    def img_alignment_sequoia(self, img_base_NIR, img_RED, width, height):
        """This class takes the five images given by Sequoia Camera and makes a photogrammetric
        alignment. Returns four images (GRE, NIR, RED, REG) aligned with the RGB image"""

        # Makes resize to all images

        base_NIR = cv2.resize(img_base_NIR, (width, height), interpolation=cv2.INTER_LINEAR)
        b_RED = cv2.resize(img_RED, (width, height), interpolation=cv2.INTER_LINEAR)

        # Makes a stabilization with NIR image like base

        stb_RED = self.estabilizador_imagen(b_RED, base_NIR)

        return base_NIR, stb_RED
#===========================================================================        
    
    def estabilizador_imagen(self, imagen_base, imagen_a_estabilizar, radio = 0.75, error_reproyeccion = 4.0, coincidencias = False):
        """Esta clase devuelve una secuencia de imágenes tomadas de la cámara estabilizada con respecto a la primera imagen"""
        
        # Se obtienen los puntos deinterés
        
        (kpsBase, featuresBase) = self.obtener_puntos_interes(imagen_base)
        (kpsAdicional, featuresAdicional) = self.obtener_puntos_interes(imagen_a_estabilizar)
        # Se buscan las coincidencias        
        
        M = self.encontrar_coincidencias(imagen_base, imagen_a_estabilizar, kpsBase, kpsAdicional, featuresBase, featuresAdicional, radio)
        
        if M is None:
            print("pocas coincidencias")
            return None
        
        if len(M) > 4:
            # construct the two sets of points
            
#            M2 = cv2.getPerspectiveTransform(ptsA,ptsB)
            (H, status) = self.encontrar_H_RANSAC_Estable(M, kpsBase, kpsAdicional, error_reproyeccion)
            estabilizada = cv2.warpPerspective(imagen_base,H,(imagen_base.shape[1],imagen_base.shape[0]))
            return estabilizada
        print("sin coincidencias")
        return None
    
    #--------------------------------------------------------------------------
    def obtener_puntos_interes(self, imagen):
        """Se obtienen los puntos de interes cn SIFT"""
        
        descriptor = cv2.xfeatures2d.SIFT_create()
        (kps, features) = descriptor.detectAndCompute(imagen, None)
        
        return kps, features
    

    #--------------------------------------------------------------------------
    def encontrar_coincidencias(self, img1, img2, kpsA, kpsB, featuresA, featuresB, ratio):
        """Metodo para estimar la homografia"""
        
        matcher = cv2.DescriptorMatcher_create("BruteForce")
        rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
        matches = []
#        
#        # loop over the raw matches
        for m in rawMatches:
#            # ensure the distance is within a certain ratio of each
#            # other (i.e. Lowe's ratio test)
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                matches.append((m[0].trainIdx, m[0].queryIdx))
        
#        print (matches)
        return matches

    #--------------------------------------------------------------------------
    def encontrar_H_RANSAC_Estable(self, matches, kpsA, kpsB, reprojThresh):
        """Metodo para aplicar una H a una imagen y obtener la proyectividad"""
        
        if len(matches) > 4:
            # construct the two sets of points
            ptsA = np.float32([kpsA[i].pt for (_, i) in matches])
            ptsB = np.float32([kpsB[i].pt for (i, _) in matches])
    
            # compute the homography between the two sets of points
            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)
            
            return (H, status)

        return None
    
    
    def ndviCalculation(self,img_RED, img_NIR, width, height):
        
        stb_NIR, stb_RED = self.img_alignment_sequoia(img_NIR, img_RED, width, height)
        
        red = np.array(stb_RED, dtype=float)
        nir = np.array(stb_NIR, dtype=float)
        
        check = np.logical_and(red > 1,nir > 1)
        
        ndvi = np.where(check, (nir-red)/(nir+red), 0)
        ndvi_index = ndvi
        
        if ndvi.min() < 0:
            ndvi = ndvi + ndvi.min()*-1
        
        ndvi = (ndvi*255)/ ndvi.max()
        ndvi = ndvi.round()
        
        ndvi_image = np.array(ndvi, dtype=np.uint8)
        
        return ndvi_index, ndvi_image
#===========================================================================        
def main():
    """Computer Vision Class"""
    print("Computer Vision Class by Jorge Martinez")

#===========================================================================
if __name__ == '__main__':
    main()

Computer Vision Class by Jorge Martinez


In [2]:
import cv2
import numpy as np
import os.path
#import serial
import math
from scipy.interpolate import interp1d
from time import time
import cv2

In [33]:
# -*- coding: utf-8 -*-
"""

@author: jolumartinez
"""
import cv2
import numpy as np
import os.path
#import serial
import math
from scipy.interpolate import interp1d
from time import time
import cv2

images_alignment = ProyectividadOpenCV()

#It defines image size
width, height = 700, 500

nombre_carpeta = "sequoia_images/"

#Reading images
img_RGB = cv2.imread(nombre_carpeta + "img_RGB.JPG",0)
img_GRE = cv2.imread(nombre_carpeta + "img_GRE.TIF",0)
img_NIR = cv2.imread(nombre_carpeta + "img_NIR.TIF",0)
img_RED = cv2.imread(nombre_carpeta + "img_RED.TIF",0)
img_REG = cv2.imread(nombre_carpeta + "img_REG.TIF",0)



merged_fix_bad = cv2.merge((img_RED, img_NIR, img_REG))
cv2.imshow('frame', merged_fix_bad)
cv2.waitKey(0)

stb_NIR, stb_RED = images_alignment.img_alignment_sequoia(img_NIR, img_RED, width, height)

merged_fix_stb = cv2.merge((stb_RED, stb_NIR, stb_RED))
cv2.imshow('frame', merged_fix_stb)
cv2.waitKey(0)

ndvi_index, ndvi_image = images_alignment.ndviCalculation(img_RED, img_NIR, width, height)
im_color = cv2.applyColorMap(ndvi_image, cv2.COLORMAP_RAINBOW)
cv2.imshow('frame', im_color)
cv2.waitKey(0)


cv2.destroyAllWindows()

In [34]:
kernel = np.ones((3,3), np.uint8) 

images_alignment = ProyectividadOpenCV()

#It defines image size
width, height = 700, 500

nombre_carpeta = ""
img_NIR = cv2.imread("IMG_0002_4.jpg")
img_RED = cv2.imread("IMG_0002_3.jpg")

img_NIR = cv2.cvtColor(img_NIR, cv2.COLOR_BGR2GRAY)
img_NIR = cv2.erode(img_NIR, kernel, iterations=1)
cv2.imshow('Gris NIR', img_NIR)
cv2.waitKey(0)

img_RED = cv2.cvtColor(img_RED, cv2.COLOR_BGR2GRAY)
img_RED = cv2.erode(img_RED, kernel, iterations=1)
cv2.imshow('Gris RED', img_RED)
cv2.waitKey(0)

ndvi_index, ndvi_image = images_alignment.ndviCalculation(img_RED, img_NIR, width, height)
im_color = cv2.applyColorMap(ndvi_image, cv2.COLORMAP_RAINBOW)
cv2.imshow('frame', im_color)
cv2.waitKey(0)



cv2.destroyAllWindows()

In [17]:
img_gris.shape

(960, 1280)

In [18]:
img = cv2.imread("IMG_0007_4.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow('gray', img)
cv2.waitKey(0)

# Taking a matrix of size 5 as the kernel 
kernel = np.ones((3,3), np.uint8) 
  
# The first parameter is the original image, 
# kernel is the matrix with which image is  
# convolved and third parameter is the number  
# of iterations, which will determine how much  
# you want to erode/dilate a given image.  
img_erosion = cv2.erode(img, kernel, iterations=1) 
  
cv2.imshow('Input', img)
cv2.waitKey(0)

cv2.imshow('Erosion', img_erosion) 
cv2.waitKey(0)


cv2.destroyAllWindows()

In [37]:
import numpy as np
import cv2

#cap = cv2.VideoCapture(1)
cap = cv2.imread("IMG_0002_4.jpg")


kernel11 = np.ones((3,),np.uint8)
kernel9 = np.ones((9,9),np.uint8)
ee3  = np.ones((3,3),np.uint8)
ee5 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(41,41))

while(True):
    # Capturar frame por frame
    #ret, frame = cap.read()
    frame = cv2.resize(cap, (450,350))

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    apertura = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel9)
    cierre = cv2.morphologyEx(apertura, cv2.MORPH_CLOSE, kernel9)
    erosion = cv2.erode(gray,kernel11,iterations = 1)
    dilatacion = cv2.dilate(gray,kernel11,iterations = 1)
    gradiente = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT,ee3)
    tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT,ee5)

    #ubicacion y tamanio de las ventanas
    cv2.imshow('gradiente',gradiente)
    cv2.imshow('cierre',cierre)
    cv2.imshow('apertura',apertura)
    cv2.imshow('erosion',erosion)
    cv2.imshow('dilatacion',dilatacion)
    cv2.imshow('tophat',tophat)

    cv2.moveWindow('gradiente', 0, 0)
    cv2.moveWindow('cierre', 500, 0)
    cv2.moveWindow('erosion',950,0)
    cv2.moveWindow('dilatacion',500,400)
    cv2.moveWindow('apertura', 0,400)
    cv2.moveWindow('tophat', 950,400)

    #evento tecla q, para el bucle
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# destruye las ventanas y libera la cam
cv2.destroyAllWindows()
